In [4]:
#!pip install tensorflow_hub tensorflow

In [60]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.client import device_lib
import os
import albumentations as A
import numpy as np

In [30]:
#tf.config.list_physical_devices('gpu')
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6087965596632455008
 xla_global_id: -1]

In [10]:
DATA_PATH = "DataCollection"
IMAGE_PATH = os.path.join(DATA_PATH, "images")
LABEL_PATH = os.path.join(DATA_PATH, "labels")

TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
VAL_PATH = os.path.join(DATA_PATH, 'val')

In [18]:
resize_transform = A.Compose([
    A.Resize(height=224, width=224, always_apply=True),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

def resize_image(img_arr, bboxes, class_labels):
    orig_img = img_arr.copy()
    return resize_transform(image=orig_img, bboxes=bboxes, class_labels=class_labels)

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = [int(coord) for coord in bbox]
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, class_names):
    img = image.copy()
    for bbox, class_name in zip(bboxes, class_names):
        img = visualize_bbox(img, bbox, class_name)
    plt.imshow(img)
    plt.show()

In [41]:
def get_annotation(path):
    tree = ET.parse(path)
    root = tree.getroot()

    label = root.find('object').find('name').text
    width = int(root.find('size')[0].text)
    height = int(root.find('size')[1].text)    

    xmin = int(root.find('object').find('bndbox').find('xmin').text)
    ymin = int(root.find('object').find('bndbox').find('ymin').text)
    xmax = int(root.find('object').find('bndbox').find('xmax').text)
    ymax = int(root.find('object').find('bndbox').find('ymax').text)
    
    coords = [xmin, ymin, xmax, ymax]
    
    return coords, label, width, height


#get_annotation(f"{os.path.join(LABEL_PATH, OBJECT_CLASS, file_name)}.xml")

In [20]:
def load_image(path):
    byte_img = tf.io.read_file(path)
    
    return tf.io.decode_jpeg(byte_img)

In [82]:
image_tensor = load_image(os.path.join(TEST_PATH, '1957ca64-6caa-11ed-a345-eb2576af4ab5.png'))
image_tensor = image_tensor[:,:,:3]
image_tensor = image_tensor / 255
image_tensor = tf.expand_dims(image_tensor, axis = 0)
image_tensor = tf.cast(image_tensor, tf.uint8)

In [83]:
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/d0/1")
detector_output = detector(image_tensor)
class_ids = detector_output["detection_classes"]

In [84]:
detector_output['detection_boxes'].numpy()[0][0]

array([0.        , 0.02784005, 0.99076486, 0.9836261 ], dtype=float32)

In [88]:
detector_output['detection_scores'][0].numpy()

array([0.07271191, 0.06280702, 0.05780304, 0.05662147, 0.0563738 ,
       0.05512178, 0.05494495, 0.05337929, 0.05296766, 0.05100016,
       0.05059639, 0.04862596, 0.04518731, 0.04123715, 0.03656029,
       0.03481734, 0.03369097, 0.03362564, 0.03267354, 0.03167859,
       0.0312239 , 0.03082477, 0.03082346, 0.03038019, 0.02912157,
       0.02880838, 0.02840948, 0.02828514, 0.02800387, 0.02759883,
       0.02615781, 0.0258301 , 0.02570098, 0.02553835, 0.02547108,
       0.02504924, 0.02461313, 0.02457962, 0.02446068, 0.0243536 ,
       0.02404318, 0.02404007, 0.02396754, 0.02322334, 0.02319351,
       0.02211283, 0.02201747, 0.02129038, 0.02090794, 0.02072514,
       0.02070938, 0.02059726, 0.019908  , 0.01977079, 0.01935735,
       0.01924396, 0.019167  , 0.01853512, 0.0184502 , 0.01813569,
       0.01781076, 0.01756942, 0.01748498, 0.01725152, 0.01720189,
       0.0171592 , 0.01695559, 0.01625103, 0.01610897, 0.01604885,
       0.01598382, 0.01582605, 0.01578198, 0.01536114, 0.01531

In [101]:
np.argmax([0.07271191, 0.06280702, 0.05780304, 0.05662147, 0.0563738])

0